In [1]:
import numpy as np
import pandas as pd
import re

In [20]:
trust = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', encoding = 'ISO-8859-1')


In [3]:
trust.head()

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [374]:
trust.columns.values

array(['PMID/PMCID', 'Publisher', 'Journal title', 'Article title',
       'COST (£) charged to Wellcome (inc VAT when charged)'], dtype=object)

In [21]:
trust.rename(columns={'COST (£) charged to Wellcome (inc VAT when charged)': 'Cost'}, inplace= True)

In [22]:
trust['PMID/PMCID'] = trust['PMID/PMCID'].astype(str)
trust['PMID/PMCID'] = trust['PMID/PMCID'].apply(lambda x: re.sub(' +',' ',str(x)))
trust['PMID/PMCID'] = trust['PMID/PMCID'].str.replace(':\n ', ':')
trust['PMID/PMCID'] = trust['PMID/PMCID'].str.replace(': ', ':')
trust['PMID/PMCID'] = trust['PMID/PMCID'].str.strip(': ')
trust['PMID/PMCID'] = trust['PMID/PMCID'].str.strip(' ')
trust['PMID/PMCID'] = trust['PMID/PMCID'].str.strip('-')
trust['PMID/PMCID'] = trust['PMID/PMCID'].str.replace('/PMC', ' PMC')
trust['PMID/PMCID'] = trust['PMID/PMCID'].str.replace('.88', '')

In [23]:
def add_pmid(x):
    
    if str(x).startswith('PMC'):
        return '0 ' + x
    
    else:
        return x
    
    
trust['PMID/PMCID'] = trust['PMID/PMCID'].apply(add_pmid)

In [24]:
all_id  = trust['PMID/PMCID'].str.split(' ', expand=True)
trust['ID_PM'] = all_id[0]
trust['ID_PMC'] = all_id[1]


In [25]:
trust['ID_PM'] = trust['ID_PM'].apply(lambda x: ''.join(list(filter(str.isdigit, str(x))))) 
#trust['ID_PMC'] = trust['ID_PMC'].apply(lambda x : ''.join(re.findall(r'\d+'  , str(x))))
trust['ID_PMC'] = trust['ID_PMC'].apply(lambda x: ''.join(list(filter(str.isdigit, str(x)))))  
#trust['ID_PM'] = trust['ID_PM'].apply(lambda x: ''.join(re.findall(r'\d+'  , str(x))))
# 23495205685.88 PMC3743159  How to handle
# What to fill for none


In [26]:
trust.ID_PM.replace('', np.nan, inplace=True)
trust.ID_PMC.replace('', np.nan, inplace=True)

In [27]:
trust['ID_PM'] = pd.to_numeric(trust.ID_PM, errors='coerce').fillna(0).astype(np.int64)
trust['ID_PMC'] = pd.to_numeric(trust.ID_PMC, errors='coerce').fillna(0).astype(np.int64)


In [12]:
trust.ID_PM.isnull().value_counts()
#pd.isnull(trust).any()
#trust.info()

False    2127
Name: ID_PM, dtype: int64

In [31]:
trust.ID_PMC.isnull().value_counts()

False    2127
Name: ID_PMC, dtype: int64

In [402]:
trust.to_csv('trust.csv')

In [414]:
#trust.iloc[43,:]
#trust.iloc[21,:]
#trust.iloc[126,:]
trust.iloc[171:173,5]
#trust.iloc[311,:]

171    0
172    0
Name: ID_PM, dtype: int64

In [28]:
# Find duplicate ID_PM
id_pm_count = trust.groupby('ID_PM')['ID_PM'].count()
id_pm_count[id_pm_count > 1]

ID_PM
0           1477
41609          2
22735079       2
Name: ID_PM, dtype: int64

In [15]:
trust.count()

PMID/PMCID       2127
Publisher        2127
Journal title    2126
Article title    2127
Cost             2127
ID_PM            2127
ID_PMC           2127
dtype: int64

In [32]:
trust[trust.ID_PM != 0  ].drop_duplicates(subset='ID_PM', keep = 'first' )


,PMID/PMCID,Publisher,Journal title,Article title,Cost,ID_PM,ID_PMC
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56,23043264,3506128
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64,23438330,3646402
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88,23438216,3601604
7,23057412 PMC3495574,ACS,Mol Pharm,Quantitative silencing of EGFP reporter gene b...,£649.33,23057412,3495574
16,22610094,AMBSB,Journal of Biological Chemistry,Annexin-1 interaction with FPR2/ALX,£265.67,22610094,0
18,23455506 PMCID:PMC3607399,American Association of Immunologists,Journal of Immunology,Regulation of Foxp3+ Inducible Regulatory T Ce...,£3108.08,23455506,3607399
19,PMID:24015914 PMC3833349,American Chemical Society,ACS Chemical Biology,Discovery of an allosteric inhibitor binding s...,£1267.76,24015914,3833349
24,22971149 PMC3466778,American Chemical Society,American Chemical Society,Characterizing the Fatty Acid Binding Site in ...,£1533.29,22971149,3466778
25,23072351,American Chemical Society,Analytical Chemistry,Amphipathic polymers facilitate the study of f...,£2082.74,23072351,0
28,23458425 PMC3630740,American Chemical Society,Bioconjugate Chemistry,"Design, synthesis and functional activity of l...",£1565.26,23458425,3630740


In [33]:
trust.count()

PMID/PMCID       2127
Publisher        2127
Journal title    2126
Article title    2127
Cost             2127
ID_PM            2127
ID_PMC           2127
dtype: int64

In [17]:
# Find duplicate ID_PMC
id_pmc_count = trust.groupby('ID_PMC')['ID_PMC'].count()
id_pmc_count[id_pmc_count > 1]

ID_PMC
0          712
3            2
3597         2
3599         2
3818         2
3173209      2
3381227      2
3401426      2
3405234      2
3528370      2
3529206      2
3597274      2
3613719      2
3647051      2
3676342      2
3708033      2
3746156      2
Name: ID_PMC, dtype: int64

In [163]:
pd.DataFrame(trust).to_csv('trust.csv')

In [74]:
trust['Publisher'] = trust['Publisher'].str.upper()

In [82]:
def change_publisher(x):
    first_letters = [ i[0] for i in str(x).split() ] 
    abbrev = "".join(first_letters)
    if str(x).startswith('ACS'):
        return 'American Chemical Society Publications'
    elif ((abbrev.startswith('AS') | abbrev.startswith('TAS')) & (abbrev.endswith('MBI') | abbrev.endswith('MB')) )| (str(x)== 'ASBMB') :
        return 'American Society for Biochemistry and Molecular Biology'
    elif (abbrev.startswith('AS') & abbrev.endswith('M'))  | (str(x).startswith('ASM')):
        return 'American Society for Microbiology'
    elif abbrev.startswith('BC'):
        return 'BioMed Central Limited'
    elif abbrev.startswith('AS') & abbrev.endswith('H'):
        return 'American Society of Haematology'
    elif abbrev.startswith('CJS'):
        return 'Cadmus Journal Services'
    elif abbrev.startswith('CUP') | str(x).startswith('CUP'):
        return 'Cambridge University Press'
    elif abbrev.startswith('CSHP'):
        return 'Cold Spring Habour Press'
    elif abbrev.startswith('COB'):
        return 'Company of Biologist'
    elif abbrev.startswith('DJ'):
        return 'Dartmouth Journal Services'
    elif (abbrev.startswith('F') & abbrev.endswith('EB')) |('FASEB' in str(x)): # FOASFEB
        return 'Federation of American Societies for Experimental Biology'
    elif abbrev.startswith('JW')  & (abbrev.endswith('S') | abbrev.endswith('SL') | abbrev.endswith('SI')):
        return 'John Wiley & Sons'
    elif abbrev =='LB':
        return 'Landes Bioscience'
    elif abbrev.startswith('MAL'):
        return 'Mary Ann Liebert'
    elif (abbrev =='OUP') | (str(x) == 'Oxford Journals') | ('OUP' in str(x)):
        return 'Oxford University Press'
    
    elif (abbrev =='IUOC') | ('IUCR' in str(x)):
        return  'International Union Of Crystallography'
    
    elif str(x).startswith('ELSEV'):
        return 'ELSEVIER'
    elif ('PNAS' in str(x)):
        return 'PNAS Author Publication'
    
    elif ('PLOS' in str(x)) | (abbrev == 'PLOS'):
        return 'Public Library of Science'
    elif str(x).startswith('PORTLAND PRESS'):
        return 'Portland Press'
    
    elif str(x).replace(' ','').startswith('MYJOVECO'): 
        return 'MY JOVE CORP'
    elif str(x).startswith('NATIONAL ACADEMY OF SCIENCE'): 
        return 'National Academy of Sciences'
    
    elif str(x).startswith('NATURE') | (abbrev =='NPG') | (str(x) == 'NPG') : 
        return 'Nature Publishing Group'
    elif str(x).startswith('SAGE') : 
        return 'SAGE'
    elif abbrev.startswith('S') & abbrev.endswith('GM') :
        return 'Society for General Microbiology'
    elif (abbrev =='SFN') | abbrev.startswith('SON') :
        return 'Society for Neuroscience'
    elif ('SPRING' in str(x)) & ('GMBH' in str(x)):
        return 'Springer - Verlag GMBH'
    elif abbrev.startswith('T&F') | abbrev.startswith('TAF') | (str(x)=='T&F'):
        return 'Taylor & Francis'
    elif abbrev.startswith('TCOB'):
        return 'The Company of Biologists'
    elif abbrev.startswith('WSS'):
        return 'Wiley Subscription Services'
    elif str(x).startswith('BMJ') : 
        return 'BMJ Group'
    elif str(x).startswith('CADMUS') : 
        return 'Cadmus Journal Services'
    elif str(x).startswith('COLD SPRING HAR') : 
        return 'Cold Spring Habour Press'
    elif str(x).startswith('FUTURE MED'):
        return 'Future Medicine'  
    elif abbrev.startswith('S') & abbrev.endswith('LB'):
        return 'Society Of Leukocyte Biology'
    elif abbrev.startswith('TES'):
        return 'The Endrocrine Society'
    elif abbrev.startswith('W') & ('BLACKWELL' in str(x)):
        return  'WILEY-BLACKWELL'   
    else:
        return str(x)#.title()
    
trust['Publisher'] = trust['Publisher'].apply(change_publisher)


In [16]:
str1 ='WLIEY-BLACKWELL' #'WILEY-BLACKWELL'#'WILEY/BLACKWELL'

#'
#first_letters = [ i[0].upper() for i in str1.split('\W') ] 

abbrev = "".join(first_letters)
print(abbrev)



W


In [65]:
print(trust['Publisher'].value_counts())

ELSEVIER                                                                             408
Public Library Of Science                                                            307
Oxford University Press                                                              197
Wiley                                                                                136
American Society for Biochemistry and Molecular Biology                              106
Springer                                                                              81
Nature Publishing Group                                                               80
BioMed Central Limited                                                                69
Wiley-Blackwell                                                                       56
Cambridge University Press                                                            32
American Chemical Society Publications                                                30
BMJ Group            

In [76]:
pd.DataFrame(trust['Publisher'].value_counts()).to_csv('trust_sep_col.csv')

In [62]:
print('THE ENDROCRINE SOCIETY'.title())

The Endrocrine Society


In [17]:
#trust['Cost'] = trust['Cost'].astype(str)
trust['Cost'] = trust['Cost'].str.strip('£')

In [96]:
trust.Cost.head()

0       0.00
1    2381.04
2     642.56
3     669.64
4     685.88
Name: Cost, dtype: object